In [ ]:
import os
import dotenv

dotenv.load_dotenv()

# Define your variable in Python
# load configuration from .env file
os.environ["APP_NAME"] = os.getenv("APP_NAME")
os.environ["PROJECT_NAME"] = os.getenv("PROJECT_NAME")
os.environ["GIT_REPO"] = os.getenv("GIT_REPO")
os.environ["CONTEXT_DIR"] = os.getenv("CONTEXT_DIR")
os.environ["RESOURCE_GROUP"] = os.getenv("RESOURCE_GROUP")
os.environ["PORT"] = os.getenv("PORT")
os.environ["API_KEY"] = os.getenv("API_KEY")


# Preperation

You might have to install the ibmcloud cli first on your system. This is how I did it for macOS

- brew install --cask ibm-cloud-cli
- ibmcloud plugin install code-engine

Generate an environment file `.env` based on the `.env-template` environment file.

# Login

In [2]:
!echo ${RESOURCE_GROUP}

tsuedbro


In [ ]:
%%bash
# print the API_KEY
ibmcloud login --apikey ${API_KEY} -r "us-south"
ibmcloud target -g ${RESOURCE_GROUP}

# Create a new IBM Cloud Code Engine application

In [ ]:
%%bash
ibmcloud ce project create --name ${PROJECT_NAME}
# Select the project (optional, if not already selected)
ibmcloud ce project select --name ${PROJECT_NAME}

# Create the application
ibmcloud ce application create \
  --name ${APP_NAME}\
  --build-source ${GIT_REPO} \
  --build-context-dir ${CONTEXT_DIR} \
  --port ${PORT} \
  --strategy dockerfile \

# Running all the tests

In [6]:
%%bash
source .venv/bin/activate
python run_tests.py all


Running all tests with coverage...
============================= test session starts ==============================
platform darwin -- Python 3.12.3, pytest-8.4.1, pluggy-1.6.0 -- /Users/thomassuedbroecker/Documents/tsuedbro/dev/learning-2024/galaxium-travels-infrastructure/booking_system_rest/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/thomassuedbroecker/Documents/tsuedbro/dev/learning-2024/galaxium-travels-infrastructure/booking_system_rest
configfile: pytest.ini
plugins: asyncio-1.1.0, anyio-4.10.0, cov-6.2.1, mock-3.14.1
asyncio: mode=Mode.STRICT, asyncio_default_fixture_loop_scope=None, asyncio_default_test_loop_scope=function
collecting ... collected 34 items

tests/test_booking_system.py::TestFlightBooking::test_book_flight_success PASSED [  2%]
tests/test_booking_system.py::TestFlightBooking::test_book_flight_user_id_name_mismatch PASSED [  5%]
tests/test_booking_system.py::TestFlightBooking::test_book_flight_user_not_found PASSED [  8%]
tests/test_booking_system.p

# Trigger Redeployment

if you have changed something in git you can use this to trigger a redeployment

In [ ]:
%%bash

ibmcloud ce project select --name ${PROJECT_NAME}

# Create the application
ibmcloud ce application update \
  --name ${APP_NAME}\
  --build-source ${GIT_REPO} \
  --build-context-dir ${CONTEXT_DIR} \
  --port ${PORT} \
  --strategy dockerfile \

# Delete an IBM Cloud Code Engine application

you should do this to avoid unnecessary cost

In [ ]:
%%bash

# Delete the application (force, no prompt)
ibmcloud ce application delete --name ${APP_NAME} --force 